In [1]:
import os
import numpy as np
import talib as ta
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [4]:
filelist = os.listdir(path)
df = pd.DataFrame()
for i in tqdm(filelist[-200:]):
    df0 = pd.read_csv(os.path.join(path, i))
    df = pd.concat([df, df0])
df

100%|██████████| 200/200 [00:13<00:00, 14.88it/s]


,1657411200000,21580.00,21594.20,21360.00,21366.60,19778.512,1657414799999,424701030.82874,174918,9041.096,...,22570.60,22656.80,22257.10,22314.80,37519.543,1658710799999,841324838.71433,291928,18206.726,408239501.19480
0,1.657415e+12,21366.7,21487.9,21330.3,21407.5,16102.491,1.657418e+12,3.449527e+08,141693.0,8004.320,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.657418e+12,21407.5,21489.0,21095.4,21300.6,34944.444,1.657422e+12,7.437294e+08,259761.0,15972.628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.657422e+12,21300.5,21338.0,21222.0,21288.6,11519.224,1.657426e+12,2.452001e+08,114596.0,5651.064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.657426e+12,21288.6,21304.0,21152.8,21268.3,18149.125,1.657429e+12,3.851330e+08,153313.0,8922.970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.657429e+12,21268.3,21365.6,21261.6,21350.0,12804.178,1.657433e+12,2.729387e+08,101539.0,6841.511,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [6]:
# AI写的方法
limit=10000
df1['close_prev'] = df1['c'].shift(1)
df1['open_prev'] = df1['o'].shift(1)

# 计算中间变量
df1['H_C1'] = np.abs(df1['h'] - df1['close_prev'])
df1['L_C1'] = np.abs(df1['l'] - df1['close_prev'])
df1['H_L'] = np.abs(df1['h'] - df1['l'])
df1['C1_O1'] = np.abs(df1['close_prev'] - df1['open_prev'])
df1['K'] = df1[['H_C1', 'L_C1']].max(axis=1)

# 计算R值
cond1 = df1['H_C1'] >= df1[['L_C1', 'H_L']].max(axis=1)
cond2 = df1['L_C1'] >= df1[['H_C1', 'H_L']].max(axis=1)

df1['R'] = np.select(
    [cond1, cond2, ~(cond1 | cond2)],
    [
        df1['H_C1'] - 0.5 * df1['L_C1'] + 0.25 * df1['C1_O1'],
        df1['L_C1'] - 0.5 * df1['H_C1'] + 0.25 * df1['C1_O1'],
        df1['H_L'] + 0.25 * df1['C1_O1']
    ],
    default=np.nan
)

# 计算SI
numerator = (df1['c'] - df1['close_prev']) + 0.5 * (df1['c'] - df1['o']) + 0.25 * (df1['close_prev'] - df1['open_prev'])
df1['SI'] = 50 * (numerator / df1['R']) * df1['K'] / limit

# 计算ASI（累积SI）
df1['g1.ASI'] = df1['SI'].cumsum()
df1 = df1[1:]
df1 = df1[['t', 'o', 'h', 'l', 'c', 'v', 'g1.ASI']]

In [7]:
df1.to_csv('ASIHdata.csv', index=False)

In [114]:
def calculate_r(row):
    H_C1 = row['H_C1']
    L_C1 = row['L_C1']
    H_L = row['H_L']
    C1_01 = row['C1_01']
    # 计算各条件下的R值
    if H_C1 >= max(L_C1, H_L):
        return H_C1 - 0.5 * L_C1 + 0.25 * C1_01
    elif L_C1 >= max(H_C1, H_L):
        return L_C1 - 0.5 * H_C1 + 0.25 * C1_01
    else:
        return H_L + 0.25 * C1_01

In [115]:
df1['t'] = df1['t'] / 1000
df1['t'] = df1['t'].astype('int')
df1['H_C1'] = abs(df1['h'] - df1['c'].rolling(2).apply(lambda x: x.iloc[0]))
df1['L_C1'] = abs(df1['l'] - df1['c'].rolling(2).apply(lambda x: x.iloc[0]))
df1['H_L'] = abs(df1['h'] - df1['l'])
df1['C1_01'] = abs(df1['c'].rolling(2).apply(lambda x: x.iloc[0]) - df1['o'].rolling(2).apply(lambda x: x.iloc[0]))
df1 = df1[1:]
df1['k'] = df1[['H_C1', 'L_C1']].max(axis=1)
df1['R'] = df1.apply(calculate_r, axis=1)
df1['result'] = 50 * (
(df1['c'] - df1['c'].rolling(2).apply(lambda x: x.iloc[0]) + 0.5 * (df1['c'] - df1['o']) + 0.25 * (
        df1['c'].rolling(2).apply(lambda x: x.iloc[0]) - df1['o'].rolling(2).apply(lambda x: x.iloc[0])))) * df1[
                    'k'] / limit
df1 = df1[1:]
df1['g1.asi'] = df1['result'].rolling(2).apply(lambda x: x.iloc[0] + x.iloc[1])

In [8]:
df1

,t,o,h,l,c,v,g1.ASI
1,1.660439e+12,24500.0,24632.6,24393.8,24573.7,23429.843,0.331565
2,1.660442e+12,24573.6,24599.5,24485.5,24585.0,8716.255,0.449536
3,1.660446e+12,24585.0,24610.0,24522.9,24554.3,7216.169,0.300414
4,1.660450e+12,24554.3,24628.1,24484.4,24555.7,11073.886,0.286824
5,1.660453e+12,24555.7,24599.4,24511.9,24570.0,6706.369,0.341169
...,...,...,...,...,...,...,...
19,1.672168e+12,16630.9,16681.3,16580.0,16681.3,16782.160,-49.093240
20,1.672171e+12,16681.3,16689.9,16651.0,16658.3,9923.424,-49.157664
21,1.672175e+12,16658.4,16714.1,16646.3,16696.3,7955.232,-48.963445
22,1.672178e+12,16696.4,16702.0,16678.5,16682.0,5519.179,-48.995901


In [9]:
df1['t'] = df1['t'] / 1000
df1['t'] = df1['t'].astype('int')
df1 = df1[['t', 'o', 'h', 'l', 'c', 'v', 'g1.ASI']]
# 获取后一个周期的值
next_value = df1['g1.ASI'].shift(-1)

# 创建三个条件的掩码
greater = df1['g1.ASI'] < next_value  # 当前值小于后一个值，说明后一个值更大
less = df1['g1.ASI'] > next_value     # 当前值大于后一个值，说明后一个值更小

# 按条件赋值
df1['judge'] = 0  # 初始化默认值为0
df1.loc[greater, 'judge'] = 1  # 后一个值更大时赋值为1
df1.loc[less, 'judge'] = -1    # 后一个值更小时赋值为-1
df1

,t,o,h,l,c,v,g1.ASI,judge
1,1660438800,24500.0,24632.6,24393.8,24573.7,23429.843,0.331565,1
2,1660442400,24573.6,24599.5,24485.5,24585.0,8716.255,0.449536,-1
3,1660446000,24585.0,24610.0,24522.9,24554.3,7216.169,0.300414,-1
4,1660449600,24554.3,24628.1,24484.4,24555.7,11073.886,0.286824,1
5,1660453200,24555.7,24599.4,24511.9,24570.0,6706.369,0.341169,1
...,...,...,...,...,...,...,...,...
19,1672167600,16630.9,16681.3,16580.0,16681.3,16782.160,-49.093240,-1
20,1672171200,16681.3,16689.9,16651.0,16658.3,9923.424,-49.157664,1
21,1672174800,16658.4,16714.1,16646.3,16696.3,7955.232,-48.963445,-1
22,1672178400,16696.4,16702.0,16678.5,16682.0,5519.179,-48.995901,1


In [10]:
df1=df1[:-1]
df1

,t,o,h,l,c,v,g1.ASI,judge
1,1660438800,24500.0,24632.6,24393.8,24573.7,23429.843,0.331565,1
2,1660442400,24573.6,24599.5,24485.5,24585.0,8716.255,0.449536,-1
3,1660446000,24585.0,24610.0,24522.9,24554.3,7216.169,0.300414,-1
4,1660449600,24554.3,24628.1,24484.4,24555.7,11073.886,0.286824,1
5,1660453200,24555.7,24599.4,24511.9,24570.0,6706.369,0.341169,1
...,...,...,...,...,...,...,...,...
18,1672164000,16673.0,16676.9,16608.4,16630.9,17174.939,-49.241342,1
19,1672167600,16630.9,16681.3,16580.0,16681.3,16782.160,-49.093240,-1
20,1672171200,16681.3,16689.9,16651.0,16658.3,9923.424,-49.157664,1
21,1672174800,16658.4,16714.1,16646.3,16696.3,7955.232,-48.963445,-1


In [11]:
df1.to_csv('ASIHdata.csv', index=False)